In [21]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

## 数据集生成&读取部分

(我们使用线性模型参数$\mathbf{w} = [w1, w2]^\top$、$b = b$
和噪声项$\epsilon$生成数据集及其标签：

$$\mathbf{y}= \mathbf{X} \mathbf{w} + b + \mathbf\epsilon.$$
)

$\epsilon$可以视为模型预测和标签时的潜在观测误差。
在这里我们认为标准假设成立，即$\epsilon$服从均值为0的正态分布。
为了简化问题，我们将标准差设为0.01。

包括两个部分：

- 数据集生成
- 数据集读取（使用 yield）

In [22]:
# 数据集生成
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

# 小批量读取数据集
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

## 模型定义部分

包括以下几个步骤：

- 定义模型
- 定义损失函数
- 定义优化算法（小批量随机梯度下降）


In [23]:
# 定义模型
def linreg(X, w, b):  #@save
    """线性回归模型"""
    return torch.matmul(X, w) + b

# 定义损失函数
#（整个batchsize样本集上的损失函数，如果要计算平均单个样本的损失函数需要除以batchsize）
def squared_loss(y_hat, y):  #@save
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

# 定义优化算法（小批量随机梯度下降）
def sgd(params, lr, batch_size):  #@save
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            # param -= lr * param.grad
            param.grad.zero_()

## 模型使用部分

主要有以下几个步骤：

- 生成随机模拟数据
- 初始化模型参数
- 训练模型
- 输出结果


In [24]:
# 1.生成随机模拟数据
true_w1, true_w2 = 4, -1.0 # 线性模型 feature1、feature2、……参数
true_b = 10 #
total_num = 1000
print(f"各参数实际值：w1: {true_w1}, w2: {true_w2}, b: {true_b}\n")

true_w = torch.tensor([true_w1, true_w2])
true_b = true_b
features, labels = synthetic_data(true_w, true_b, total_num)

# 2.初始化模型参数
# w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
# b = torch.zeros(1, requires_grad=True)

## exercise-1:
w = torch.tensor([[0], [0]], dtype=torch.float, requires_grad=True)
b = torch.tensor([0], dtype=torch.float, requires_grad=True)

print(w)
print(b)

# 3.训练模型
# lr = 0.0001 # 学习率
lr = 1 # 学习率
batch_size = 10 # 小样本数
num_epochs = 5 # 训练迭代次数
net = linreg # 网络模型
loss = squared_loss # 损失函数

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # X和y的小批量损失
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

# 输出结果
print(f'w的估计值: {w.reshape(true_w.shape)}')
print(f'b的估计值: {b}')
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

各参数实际值：w1: 4, w2: -1.0, b: 10

tensor([[0.],
        [0.]], requires_grad=True)
tensor([0.], requires_grad=True)
epoch 1, loss 0.000075
epoch 2, loss 0.000077
epoch 3, loss 0.000059
epoch 4, loss 0.000073
epoch 5, loss 0.000082
w的估计值: tensor([ 3.9925, -1.0013], grad_fn=<ReshapeAliasBackward0>)
b的估计值: tensor([9.9979], requires_grad=True)
w的估计误差: tensor([0.0075, 0.0013], grad_fn=<SubBackward0>)
b的估计误差: tensor([0.0021], grad_fn=<RsubBackward1>)
